## what is <strong>ATTENTION</strong>
in a model to perform well, it should be able to focus on the most relevant parts of the input.which is called <strong>attention</strong>.
for example in a sentence like "A cat sat on the mat and jumped over the dog", the model should be able to focus on the words "cat", "sat", "on", "the", "mat" and "jumped" and etc. By doing so it makes it understand that the actual sentence is about a cat jumping over dog not a dog jumping over cat or anything else.
- it is essential for language translation models to learn how to focus on the most relevant parts of the input.
- for exmple we are converting timi sanchai xau,bhai? to english it cant be you fine are,brother? but you are fine,bhai?
- here needs attention mechanism.
- now in transformers, the model uses attention to focus on the most relevant parts of the input.but before transformers,there used to be RNNs which are neural network + encoder decoder which encoder generated context vector and decoder used that context vector to generate the output.
- Attention Mechanism are 
  1.simplified attention
  2.multihead attention
  3.self attention

## now lets understand RNNs and How they work.
1. RNNs are a type of neural network that is used to process sequential data.
2. when a single word comes in encoder in encoder a hidden state is generated.
3. and it continuously updated until the end of the sentence.
4. then in final hidden state there is context of the whole sentence.which is kind of memory of the sentence.
5. this final hidden state is used by the decoder to generate the output.
6. the output is generated as one word at a time using the context which now helps generating better output.
- But there was problem with this that the decoder would use only the final hidden state to generate the output. which could be a problem if the sentence is long.
- to solve this problem transformers introduced attention mechanism.

# Before Transformers there was Bahdanau attention which would modify the RNNs in which the decoder would use the encoder previous hidden state to generate the output.
- for example there is a sentence like "A cat sat on the mat and jumped over the dog".
  while decoder is translating word jumped,it may give attention to the word cat also whch makes model understand that the actual sentence is about a cat jumping over dog not a dog jumping over cat or anything else.
- this is called Bahdanau attention.
* upto now we have understood how decoding happens using the encoders previous hidden state to generate the next word.
* Lets understand What is self-Attention.
  - Self attention is technique in which the mechanism's ability to weigh attention weight within the single input sequence. 
  

# simplified Attention mechanism

- for example consider we have a sintence that "Your Journey Starts with a Question".
- after vector and input embedding we get for each word/token a vector embedding a long dimensional vector.
- now these vectors while plotting would give us a graph in visual representation.
- now to get the context vector for a token with any other token we need to calculate the similarity between the two vectors.
- this similarity is given by the dot product of the two vectors.
  ![attention](./atten.png)

  

In [3]:
import torch 

In [24]:
def attention_calculation(query,input):
    # print(input.shape)
    attention_scores = torch.empty(len(input))
    for i in range(len(input)):
        # print(len(input))
        attention_scores[i] = torch.dot(query, input[i])
    return attention_scores


In [23]:
# coding in python
inputs = torch.tensor(
    [
        [0.1, 0.2, 0.3, 0.4],
        [0.2, 0.3, 0.4, 0.5],
        [0.3, 0.4, 0.5, 0.6],
        [0.4, 0.5, 0.6, 0.7],
        [0.5, 0.6, 0.7, 0.8],
        [0.6, 0.7, 0.8, 0.9],
    ]
)
# print(inputs[1])

attention_score = attention_calculation(inputs[1],inputs)#calculates attention score with query and all the input tokens but we need to now normalize it to get better results later for use

print(attention_score)

torch.Size([6, 4])
tensor([0.4000, 0.5400, 0.6800, 0.8200, 0.9600, 1.1000])


## normalization makes it between 0 and 1.
- one way is to divide each score by the sum of all scores 
- but we havve problem because we dont want to consider a token attention if it is not near to the query token.
- we use softmax to normalize the scores
- in this way we get the attention score between 0 and 1 for each token by exponential of atention score divided by sum of all attention scores which make model understand that no need to give attention to a token if it is not near to the query token 


In [17]:
def soft_max_naive(x):
    return torch.exp(x)/torch.exp(x).sum(dim=0)

normalized_attention_score = soft_max_naive(attention_score)
print(normalized_attention_score)
print(normalized_attention_score.sum(dim=0))

tensor([0.1142, 0.1313, 0.1510, 0.1737, 0.1999, 0.2299])
tensor(1.)


In [10]:
#it is better to use pytorch softmax function

normalized_attention_score_2 = torch.softmax(attention_score, dim=0)
print(normalized_attention_score_2)
print(normalized_attention_score_2.sum(dim=0))

tensor([0.1142, 0.1313, 0.1510, 0.1737, 0.1999, 0.2299])
tensor(1.)


# now we want to calculate the context vector
- it is calculated by multiplying the normalized attention score with the input tokens's input embedding vector
- then after multiplying we get a vector of same dimension as the input embedding vector for each tokens of the input sequence which is context vector for the particular query with tokens of input sequence. 
- now these context vectors are summed to get the final context vector for the query token.
- now the final context vector is used to generate the output by using the decoder.
  ![context](./cntxt.png)
![attention](./ctxt.png)

In [ ]:
#code 
def calculate_context_vector(inputs, normalized_attention_score_2):
    context_vector = torch.empty(inputs.shape[1])
    for i,x_i in enumerate(inputs):
        context_vector+= normalized_attention_score_2[i] * x_i
    return context_vector


context_vector = calculate_context_vector(inputs, normalized_attention_score_2)
print(context_vector)#this is the context vector for the query token 'Journey'

tensor([1.2103, 1.0927, 0.5903, 0.6903])


In [45]:
attention = inputs @ inputs.T
print(attention)
normalized_attention = torch.softmax(attention, dim=-1)
print(normalized_attention)

tensor([[0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000],
        [0.4000, 0.5400, 0.6800, 0.8200, 0.9600, 1.1000],
        [0.5000, 0.6800, 0.8600, 1.0400, 1.2200, 1.4000],
        [0.6000, 0.8200, 1.0400, 1.2600, 1.4800, 1.7000],
        [0.7000, 0.9600, 1.2200, 1.4800, 1.7400, 2.0000],
        [0.8000, 1.1000, 1.4000, 1.7000, 2.0000, 2.3000]])
tensor([[0.1279, 0.1414, 0.1562, 0.1727, 0.1908, 0.2109],
        [0.1142, 0.1313, 0.1510, 0.1737, 0.1999, 0.2299],
        [0.1014, 0.1214, 0.1454, 0.1740, 0.2083, 0.2494],
        [0.0897, 0.1118, 0.1393, 0.1735, 0.2163, 0.2695],
        [0.0790, 0.1025, 0.1329, 0.1723, 0.2235, 0.2899],
        [0.0693, 0.0935, 0.1262, 0.1704, 0.2300, 0.3105]])


In [46]:
row_2 = normalized_attention[1]
print(row_2)
print(row_2.sum(dim=0))

tensor([0.1142, 0.1313, 0.1510, 0.1737, 0.1999, 0.2299])
tensor(1.)


In [48]:
#calculate context vector for each query token
context_vector = normalized_attention @ inputs
print(context_vector)

tensor([[0.3790, 0.4790, 0.5790, 0.6790],
        [0.3903, 0.4903, 0.5903, 0.6903],
        [0.4015, 0.5015, 0.6015, 0.7015],
        [0.4123, 0.5123, 0.6123, 0.7123],
        [0.4228, 0.5228, 0.6228, 0.7228],
        [0.4330, 0.5330, 0.6330, 0.7330]])


## How to convert Input Embedding to Context Vector ??
- it is done by 1st we calculate the attention score for each token of the input sequence.
- this is done by calculating the dot product of the query token with each token of the input sequence.
- then we normalize the attention score using softmax.
- then we calculate the context vector by 1st multiplying the normalized attention score of each token (with respect to query token) with the input embedding vector of each token.
- then we sum all these context vectors to get the final context vector for the query token.
- this is fed to the decoder to generate the output.

# now how do we even get the attention score for each token of the input sequence and the context vector?
- to get attention score with each other in a sequence we do matrix multiplication of input embedding matrix to transpose of input embedding matrix.which returns the attention score for each token of the input sequence with respect to each other token of the input sequence.
- now to get the normalized attention score we use softmax.
- now to get the context vector for each token of the input sequence we do matrix multiplication of normalized attention score with input embedding matrix.
* boom we get the context vector for each token of the input sequence.that 1st row of this vectr gives the context vector for the 1st token and so on... also remember that 1st row of the input embedding matrix gives the input embedding vector for the 1st token.so we just converted all the input embedding vectors to context vectors.

** REMEMBER THAT WE NEED TO USE TRAINABLE WEIGHTS FOR THIS. **